In [1]:
import os
import requests
from requests.exceptions import HTTPError, ConnectionError, Timeout, RequestException
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Annotated, List, Optional, Any, Dict
from pydantic import BaseModel, HttpUrl, Field, ValidationError
from datetime import date
from langchain_core.messages import BaseMessage, AIMessage, ToolMessage

_ = load_dotenv()

In [2]:
class APIError(Exception):
    """Exceção base para erros da API."""
    pass

class RequestFailedError(APIError):
    """Exceção para falhas na requisição HTTP (status 4xx ou 5xx)."""
    pass

class DataProcessingError(APIError):
    """Exceção para falhas no processamento ou validação dos dados da API."""
    pass

class NotFoundError(RequestFailedError):
    """Exceção específica para recursos não encontrados (status 404)."""
    pass

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"


class Link(BaseModel):
    """Modelo para representar links de navegação da API."""
    href: HttpUrl = Field(..., description="A URL do recurso vinculado.")
    rel: str = Field(..., description="O tipo da relação do link (ex: 'self', 'next', 'first').")
    type: Optional[str] = Field(None, description="O tipo de mídia do recurso vinculado, se disponível.")

class DeputadoBusca(BaseModel):
    """Modelo para representar um deputado em uma lista de busca (endpoint /deputados)."""
    id: int = Field(..., description="O ID numérico único do deputado.")
    uri: HttpUrl = Field(..., description="A URI para obter informações detalhadas deste deputado.")
    nome: str = Field(..., description="O nome parlamentar do deputado.")
    siglaPartido: str = Field(..., description="A sigla do partido ao qual o deputado é filiado.")
    uriPartido: HttpUrl = Field(..., description="A URI para obter informações sobre o partido do deputado.")
    siglaUf: str = Field(..., description="A sigla da Unidade Federativa pela qual o deputado foi eleito.")
    urlFoto: HttpUrl = Field(..., description="A URL da foto oficial do deputado.")
    email: Optional[str] = Field(None, description="O endereço de e-mail oficial do deputado, se disponível.")

class DeputadosBuscaResponse(BaseModel):
    """Modelo da resposta completa do endpoint GET /deputados."""
    dados: List[DeputadoBusca] = Field(..., description="Uma lista de objetos 'DeputadoBusca' encontrados.")
    links: List[Link] = Field(..., description="Links de paginação e navegação da resposta da API.")

class Gabinete(BaseModel):
    """Modelo para os dados do gabinete de um deputado."""
    andar: Optional[str] = Field(None, description="O andar onde o gabinete está localizado, se disponível.")
    email: Optional[str] = Field(None, description="O endereço de e-mail do gabinete, se disponível.")
    nome: Optional[str] = Field(None, description="O nome do gabinete, se disponível.")
    predio: Optional[str] = Field(None, description="O prédio onde o gabinete está localizado, se disponível.")
    sala: Optional[str] = Field(None, description="O número da sala do gabinete, se disponível.")

class UltimoStatusDeputado(BaseModel):
    """Modelo para o último status de exercício parlamentar de um deputado."""
    condicaoEleitoral: str = Field(..., description="A condição eleitoral atual do deputado.")
    data: date = Field(..., description="A data do último registro de status.")
    descricaoStatus: Optional[str] = Field(None, description="Uma descrição do último status do deputado, se disponível.")
    email: Optional[str] = Field(None, description="O endereço de e-mail do deputado no último status, se disponível.")
    gabinete: Gabinete = Field(..., description="Informações detalhadas sobre o gabinete do deputado.")
    nome: str = Field(..., description="O nome parlamentar do deputado no último status.")
    siglaPartido: str = Field(..., description="A sigla do partido do deputado no último status.")
    siglaUf: str = Field(..., description="A sigla da Unidade Federativa do deputado no último status.")
    uri: HttpUrl = Field(..., description="A URI para o recurso do deputado no último status.")
    uriPartido: Optional[HttpUrl] = Field(None, description="A URI para o recurso do partido do deputado no último status, se disponível.")
    urlFoto: HttpUrl = Field(..., description="A URL da foto do deputado no último status.")

class DeputadoDetalhesDados(BaseModel):
    """Modelo para os dados detalhados de um deputado (endpoint /deputados/{id})."""
    cpf: str = Field(..., description="O número de CPF do parlamentar.")
    dataFalecimento: Optional[date] = Field(None, description="A data de falecimento do parlamentar, se aplicável.")
    dataNascimento: date = Field(..., description="A data de nascimento do parlamentar.")
    escolaridade: str = Field(..., description="O nível de escolaridade declarado pelo parlamentar.")
    id: int = Field(..., description="O ID numérico único do parlamentar.")
    municipioNascimento: str = Field(..., description="O município de nascimento do parlamentar.")
    nomeCivil: str = Field(..., description="O nome civil completo do parlamentar.")
    redeSocial: List[str] = Field(..., description="Uma lista de URLs de redes sociais do parlamentar.")
    sexo: str = Field(..., description="O gênero do parlamentar ('M' para masculino, 'F' para feminino).")
    ufNascimento: str = Field(..., description="A sigla da Unidade Federativa de nascimento do parlamentar.")
    ultimoStatus: UltimoStatusDeputado = Field(..., description="O último status de exercício parlamentar do deputado.")

class DeputadoDetalhesResponse(BaseModel):
    """Modelo da resposta completa do endpoint GET /deputados/{id}."""
    dados: DeputadoDetalhesDados = Field(..., description="O objeto 'DeputadoDetalhesDados' contendo todas as informações do deputado.")
    links: List[Link] = Field(..., description="Links de navegação da resposta da API.")

class ObterDetalhesDeputadoInput(BaseModel):
    """Esquema de entrada para a ferramenta obter_detalhes_deputado."""
    id_deputado: int = Field(..., description="O ID numérico único do deputado.")

def _make_request(url: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    Função auxiliar para fazer requisições HTTP à API da Câmara dos Deputados.

    Args:
        url (str): A URL completa do endpoint da API.
        params (Optional[Dict[str, Any]]): Um dicionário de parâmetros de consulta para a requisição.

    Returns:
        Dict[str, Any]: O conteúdo JSON da resposta da API.

    Raises:
        RequestFailedError: Se a requisição HTTP falhar (código de status 4xx ou 5xx).
        NotFoundError: Se o recurso solicitado não for encontrado (código de status 404).
        RequestException: Para outros erros inesperados durante a requisição.
    """
    headers = {"Accept": "application/json"}
    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        
        if response.status_code == 404:
            raise NotFoundError(f"Recurso não encontrado na URL: {url}")
        
        response.raise_for_status() # Levanta um HTTPError para status de erro (4xx ou 5xx)
        return response.json()
    except HTTPError as http_err:
        raise RequestFailedError(f"Erro HTTP ocorrido: {http_err} - Resposta: {response.text}")
    except (ConnectionError, Timeout) as e:
        raise RequestException(f"Erro de conexão ou tempo limite excedido: {e}")
    except RequestException as req_err:
        raise RequestException(f"Um erro inesperado ocorreu durante a requisição: {req_err}")

# @tool
def buscar_deputados_por_nome(nome: str) -> DeputadosBuscaResponse:
    """
    Busca por deputados cujo nome parlamentar contenha a string fornecida.

    Este método utiliza o endpoint GET /deputados da API de Dados Abertos da Câmara
    dos Deputados para filtrar parlamentares por nome.

    Args:
        nome (str): O nome completo ou parte do nome parlamentar a ser buscado.
                    A busca é insensível a maiúsculas/minúsculas e pode retornar
                    múltiplos resultados se o nome não for exato.

    Returns:
        DeputadosBuscaResponse: Um objeto Pydantic contendo uma lista de
                                deputados que correspondem ao nome fornecido.
                                Cada item na lista inclui informações básicas
                                como ID, nome, partido, UF e URL da foto.

    Raises:
        RequestFailedError: Se a requisição à API falhar devido a um erro HTTP (4xx/5xx).
        DataProcessingError: Se houver um erro na validação dos dados retornados
                             pela API contra o modelo Pydantic.
        APIError: Para outros erros relacionados à comunicação com a API.
    """
    url = f"{BASE_URL}/deputados"
    params = {"nome": nome}
    try:
        data = _make_request(url, params)
        return DeputadosBuscaResponse.model_validate(data)
    except ValidationError as e:
        raise DataProcessingError(f"Erro na validação Pydantic da resposta de busca de deputados: {e}")
    except (RequestFailedError, RequestException) as e:
        raise APIError(f"Falha ao buscar deputados por nome: {e}")

# @tool
def obter_detalhes_deputado(id_deputado: int) -> str:
    """
    Obtém todas as informações detalhadas de um deputado específico a partir de seu ID.

    Este método utiliza o endpoint GET /deputados/{id} da API de Dados Abertos da
    Câmara dos Deputados para recuperar dados cadastrais e o último status
    de exercício parlamentar de um deputado.

    Args:
        id_deputado (int): O ID numérico único do deputado a ser detalhado.
                           Este ID pode ser obtido através da função
                           'buscar_deputados_por_nome'.

    Returns:
        DeputadoDetalhesResponse: Um objeto Pydantic contendo todas as informações
                                  detalhadas do deputado, como nome civil, data de
                                  nascimento, escolaridade, dados do gabinete,
                                  e seu último status parlamentar.

    Raises:
        NotFoundError: Se nenhum deputado for encontrado com o ID fornecido (erro 404).
        RequestFailedError: Se a requisição à API falhar devido a um erro HTTP (4xx/5xx).
        DataProcessingError: Se houver um erro na validação dos dados retornados
                             pela API contra o modelo Pydantic.
        APIError: Para outros erros relacionados à comunicação com a API.
    """
    url = f"{BASE_URL}/deputados/{id_deputado}"
    try:
        data = _make_request(url)
        return DeputadoDetalhesResponse.model_validate(data)
    except ValidationError as e:
        raise DataProcessingError(f"Erro na validação Pydantic dos detalhes do deputado: {e}")
    except (RequestFailedError, RequestException) as e:
        raise APIError(f"Falha ao obter detalhes do deputado {id_deputado}: {e}")

In [3]:
print("--- Exemplo de Busca e Detalhes de Deputados ---")

nome_busca = "Bolsonaro"
try:
    print(f"\nBuscando deputados com o nome '{nome_busca}'...")
    deputados_encontrados = buscar_deputados_por_nome(nome_busca)

    if deputados_encontrados.dados:
        print(f"Encontrados {len(deputados_encontrados.dados)} resultado(s):")
        for dep_basico in deputados_encontrados.dados:
            print(f"  - ID: {dep_basico.id}, Nome: {dep_basico.nome}, Partido: {dep_basico.siglaPartido}-{dep_basico.siglaUf}")
            
            # Exemplo 2: Obter detalhes do primeiro deputado encontrado
            print(f"  Obtendo detalhes para o deputado ID: {dep_basico.id} ({dep_basico.nome})...")
            detalhes_deputado = obter_detalhes_deputado(dep_basico.id)
            
            print(f"    Nome Civil: {detalhes_deputado.dados.nomeCivil}")
            print(f"    Data de Nascimento: {detalhes_deputado.dados.dataNascimento} ({detalhes_deputado.dados.sexo})")
            print(f"    Escolaridade: {detalhes_deputado.dados.escolaridade}")
            print(f"    Partido Atual: {detalhes_deputado.dados.ultimoStatus.siglaPartido}-{detalhes_deputado.dados.ultimoStatus.siglaUf}")
            print(f"    Email do Gabinete: {detalhes_deputado.dados.ultimoStatus.gabinete.email}")
            print("-" * 40)
    else:
        print(f"Nenhum deputado encontrado com o nome '{nome_busca}'.")

except NotFoundError as e:
    print(f"Erro: {e}. Certifique-se de que o ID do deputado está correto se estiver buscando por um ID específico.")
except APIError as e:
    print(f"Erro na API: {e}")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

--- Exemplo de Busca e Detalhes de Deputados ---

Buscando deputados com o nome 'Bolsonaro'...
Encontrados 1 resultado(s):
  - ID: 92346, Nome: Eduardo Bolsonaro, Partido: PL-SP
  Obtendo detalhes para o deputado ID: 92346 (Eduardo Bolsonaro)...
    Nome Civil: EDUARDO NANTES BOLSONARO
    Data de Nascimento: 1984-07-10 (M)
    Escolaridade: Superior
    Partido Atual: PL-SP
    Email do Gabinete: dep.eduardobolsonaro@camara.leg.br
----------------------------------------


In [5]:
def executar_analise(pergunta: str):
    """Executa o agente e imprime o stream de respostas de forma legível."""
    inputs = {"messages": [("user", pergunta)]}
    print(f"\n--- Pergunta: {pergunta} ---")
    
    # Itera sobre cada passo do grafo para uma impressão mais detalhada
    for step_output in graph.stream(inputs):
        for node_name, state_update in step_output.items():
            print(f"--- Nó: {node_name} ---")
            
            # Itera sobre as mensagens no estado atual
            for message in state_update.get("messages", []):
                if isinstance(message, AIMessage):
                    print(f"  🤖 Resposta do Agente:")
                    if message.content:
                        print(f"    - Conteúdo: {message.content}")
                    if message.tool_calls:
                        print("    - Chamada de Ferramenta:")
                        for tool_call in message.tool_calls:
                            print(f"      -> Nome da Ferramenta: {tool_call['name']}")
                            print(f"      -> Argumentos: {tool_call['args']}")
                
                elif isinstance(message, ToolMessage):
                    print(f"  🛠️ Execução de Ferramenta:")
                    print(f"    - Ferramenta: {message.name}")
                    print(f"    - Status: {message.status.upper()}")
                    print(f"    - Resultado:")
                    # Impressão do resultado formatado
                    resultado = message.content
                    if resultado.startswith("Erro"):
                        print(f"      ❌ {resultado}")
                    else:
                        for linha in resultado.split('\n'):
                            print(f"      ✅ {linha}")
                    
            print("-" * 40)

In [10]:
executar_analise("Quais as informações do deputado jorge?")


--- Pergunta: Quais as informações do deputado jorge? ---
--- Nó: chatbot ---
  🤖 Resposta do Agente:
    - Chamada de Ferramenta:
      -> Nome da Ferramenta: buscar_deputados_por_nome
      -> Argumentos: {'nome': 'jorge'}
----------------------------------------
--- Nó: tools ---
  🛠️ Execução de Ferramenta:
    - Ferramenta: buscar_deputados_por_nome
    - Status: SUCCESS
    - Resultado:
      ✅ dados=[DeputadoBusca(id=205550, uri=HttpUrl('https://dadosabertos.camara.leg.br/api/v2/deputados/205550'), nome='Jorge Braz', siglaPartido='REPUBLICANOS', uriPartido=HttpUrl('https://dadosabertos.camara.leg.br/api/v2/partidos/37908'), siglaUf='RJ', urlFoto=HttpUrl('https://www.camara.leg.br/internet/deputado/bandep/205550.jpg'), email='dep.jorgebraz@camara.leg.br'), DeputadoBusca(id=178857, uri=HttpUrl('https://dadosabertos.camara.leg.br/api/v2/deputados/178857'), nome='Jorge Solla', siglaPartido='PT', uriPartido=HttpUrl('https://dadosabertos.camara.leg.br/api/v2/partidos/36844'), siglaUf

In [8]:
# out = llm_with_tools.invoke("Quem descobriu o Brasil?")

# out